In [1]:

print("Begin init")
%run -i init_notebook.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from structlog import get_logger
from tqdm import tqdm

from src.model.StockModel import StockModel
from src.repository.FileOhlcRepository import FileOhlcRepository
from src.repository.SymbolRepository import SymbolRepository
from src.scrapper.YahooScrapper import YahooScrapper
from datetime import datetime, timedelta
print("Loading modules complete", datetime.now())

Begin init


Loading modules complete 2021-06-01 14:05:14.802657


In [ ]:
from src.data_gen.dataset_generator import get_1min_data
stock = StockModel.parse("NASDAQ:AAPL")
min1_df = get_1min_data(stock)
min1_df.info()

In [ ]:
from src.calulation.utils import calculate_scalping_for_long_fast, calculate_scalping_for_short_fast

df = min1_df.copy()
df['profit'] = calculate_scalping_for_long_fast(min1_df, 0.5/100, 0.25/100, 30)
positive_count = len(df[df['profit']==1])
all_count = len(df)
print(positive_count/all_count,  positive_count, all_count)

In [36]:
from src.calulation.utils import normilize_data_
from src.data_gen.dataset_generator import get_inidicators

from src.calulation.indicator_builder import build_indicatrors

indicators = get_inidicators()


df = min1_df.tail(10000).copy()
basic_price = df.iloc[0].low
basic_volume = df.iloc[0].volume
normilize_data_(df, basic_price, basic_volume)

min1_df_with_indicators = build_indicatrors(df, indicators)
min1_df_with_indicators = min1_df_with_indicators.iloc[7000:-1] # drop warmup

min1_df_with_indicators = min1_df_with_indicators.round(5)
print(min1_df_with_indicators.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2999 entries, 2021-05-11 15:50:00 to 2021-05-25 19:58:00
Data columns (total 49 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   date                                              2999 non-null   datetime64[ns]
 1   low                                               2999 non-null   float64       
 2   close                                             2999 non-null   float64       
 3   open                                              2999 non-null   float64       
 4   high                                              2999 non-null   float64       
 5   volume                                            2999 non-null   float64       
 6   vortex_positive(n:14)                             2999 non-null   float64       
 7   vortex_negative(n:14)                             2999 non-null   float64     

In [38]:
import src.calulation.utils as ta_utils
ta_utils.fit_scalers(min1_df_with_indicators)
print("Fitting complete")


100%|██████████| 49/49 [00:00<00:00, 449.34it/s]

Fitting complete


In [ ]:
# import src.calulation.utils as ta_utils
# min1_df_with_indicators = ta_utils.scaler_df(min1_df_with_indicators)
# print("Scaling complete")

In [ ]:
# print(min1_df_with_indicators.iloc[-1].to_dict())
raw_columns = {'date', 'low', 'close', 'high', 'open', 'volume'}
for col in min1_df_with_indicators.columns:
    if col in raw_columns:
        continue

    print(f"{col}")
    plt.title(col)
    min1_df_with_indicators[col].hist(bins=100)
    plt.show()
    print(f"Min:{min1_df_with_indicators[col].min()}")
    print(f"Max:{min1_df_with_indicators[col].max()}")
    abs = np.abs(min1_df_with_indicators[col])
    print()
    print("mean", np.mean(abs))
    print("75%", np.percentile(abs, 75))
    print("80%", np.percentile(abs, 80))
    print("90%", np.percentile(abs, 99))
    print("99%", np.percentile(abs, 99))
    print("*"*100)

In [ ]:
d = min1_df_with_indicators.copy()
d.reset_index(inplace=True, drop=True)
d[['tema(n=5)','tema(n=16)', 'tema(n=32)', 'tema(n=64)', 'tema(n=128)', 'tema(n=256)', 'tema(n=512)']].plot()

In [ ]:
# min1_df_with_indicators['profit'], min1_df_with_indicators['loss'] = ta_utils.get_long_profit_and_loss(min1_df_with_indicators, l=15)


In [ ]:
# min1_df_with_indicators['profit'].hist(bins=100)
# plt.show()
#
# min1_df_with_indicators['loss'].hist(bins=100)
# plt.show()

In [ ]:
# positive = min1_df_with_indicators[min1_df_with_indicators['profit']>0.25/100]
# print(100*len(positive)/len(min1_df_with_indicators))
#
# positive = min1_df_with_indicators[min1_df_with_indicators['loss']>0.5/100]
# print(100*len(positive)/len(min1_df_with_indicators))
#
#
# print(100* len(positive) / len(all_y))